In [1]:
import numpy as np
from opensimplex import OpenSimplex
import pyqtgraph.opengl as gl
from pyqtgraph.Qt import QtCore, QtGui
import struct
import pyaudio as pa 
import sys


class AudioVizualizer(object):
    def __init__(self):
        
        
        #setup fereastra
        self.app = QtGui.QApplication(sys.argv) 
        self.window = gl.GLViewWidget()
        self.window.setWindowTitle('Audio Visualizer')
        self.window.setGeometry(0, 110, 1920, 1080)
        self.window.setCameraPosition(distance=100, elevation=50)
        self.window.show()
        
        self.nsteps = 1
        self.offset = 0 #animatie zgomot
        
        #nr puncte matrice
        self.xp = np.arange(-20, 20, self.nsteps)
        self.yp = np.arange(-20, 20, self.nsteps)
        self.nrfete = len(self.yp)
        
        #parametrii inregistrare
        self.RATE = 44100
        self.CHUNK = len(self.yp)
        self.CHUNK = len(self.xp) * len(self.yp)
        
        #inregistrare
        self.p = pa.PyAudio()
        self.stream = self.p.open(
            format = pa.paInt16,
            channels = 1,
            rate = self.RATE,
            input=True,
            output=False,
            frames_per_buffer=self.CHUNK,
        )
        
        #zgomot perlin
        self.noise = OpenSimplex()
        
        #construire parametrii plasa
        verts, faces, colors = self.plasa()
        
        #creeare plasa
        self.mesh = gl.GLMeshItem(
            faces=faces,
            vertexes=verts,
            faceColors=colors,
            drawEdges=False,
            smooth=True,
        )
        
        #caracteristici de culoare 
        self.mesh.setGLOptions('additive')
        
        #adaugare plasa in obiectul fereastra
        self.window.addItem(self.mesh)
    
    
    def plasa(self, offset=0, data=None):
        if data is not None: 
            #conversie bytes in string
            data = struct.unpack(str(self.CHUNK) + 'h', data)
            data = np.array(data, dtype='h')
            
            #datele se scaleaza pentru un efect mai bun de vizualizare
            data = data*0.003
            #vectorul data devine o matrice
            data = data.reshape((len(self.xp), len(self.yp)))
            
        #la initializarea obiectului de tip AudioVisualizer se creeaza vectorul de date
        else:
            data = np.array([1] * 1600)
            data = data.reshape((len(self.xp), len(self.yp)))

        fete = []
        culori = []
        varfuri = np.array([
            [
                x, y,data[i][j] *  self.noise.noise2(x = i/100 + offset, y = j/100 + offset)
            ] for i, x in enumerate(self.xp) for j, y in enumerate(self.yp)
        ], dtype=np.float32)

        for i in range(self.nrfete - 1):
            off = i * self.nrfete
            for j in range(self.nrfete - 1):
                fete.append([
                    j + off,
                    j + off + self.nrfete,
                    j + off + self.nrfete + 1,
                ])
                fete.append([
                    j + off,
                    j + off + 1,
                    j + off + self.nrfete + 1,
                ])
                culori.append([
                   1, 1, 1, 0.1
                ])
                culori.append([
                   1, 1, 1, 0.1
                ])
                

        fete = np.array(fete, dtype=np.uint32)
        culori = np.array(culori, dtype=np.float32)

        return varfuri, fete, culori
        
        
    def update(self):
        #citeste datele undei sonore
        data = self.stream.read(self.CHUNK, exception_on_overflow=False)
        
        #se actualizeaza parametrii plasei
        varfuri, fete, culori = self.plasa(offset=self.offset, data=data)
        self.mesh.setMeshData(vertexes=varfuri, faces=fete, faceColors=culori)
        self.offset += 0.05
        

    def start(self):
        if (sys.flags.interactive != 1) or not hasattr(QtCore, 'PYQT_VERSION'):
            QtGui.QApplication.instance().exec_()
            
            
    def animatie(self):
        timer = QtCore.QTimer()
        timer.timeout.connect(self.update)
        timer.start(30)
        self.start()
        self.update()


if __name__ == '__main__':
    t = AudioVizualizer()
    t.animatie()